# **실습 3-2 : Char-RNN**



## **Import Module**

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

## **DataSet**

### 학습할 문장 만들기

In [2]:
# 학습할 문장
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea."
            "if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 360


### 입력 문자열과 타겟 문자 준비

#### Char dictionary 만들기

In [3]:
# make charater dictionary 
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("CHARACTERS: ")
print (len(char_set))
print (char_set)
print ("DICTIONARY: ")
print (len(char_dic))
print (char_dic)

CHARACTERS: 
25
['e', 'w', 'k', 'l', ',', 'b', 'h', 'u', '.', 'a', 'g', 'd', ' ', 's', 'i', 't', 'r', 'c', 'p', 'o', 'f', "'", 'n', 'y', 'm']
DICTIONARY: 
25
{'e': 0, 'w': 1, 'k': 2, 'l': 3, ',': 4, 'b': 5, 'h': 6, 'u': 7, '.': 8, 'a': 9, 'g': 10, 'd': 11, ' ': 12, 's': 13, 'i': 14, 't': 15, 'r': 16, 'c': 17, 'p': 18, 'o': 19, 'f': 20, "'": 21, 'n': 22, 'y': 23, 'm': 24}


#### 문자열

In [4]:
data_dim    = len(char_set) # train data X:input
num_classes = len(char_set) # trian data Y:target
sequence_length = 10        # any arbitrary number
print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 25
num_classes : 25


In [5]:
dataX = []  # input sequence list
dataY = []  # target sequence list 

# we will make 170 sequences 
for i in range(0, len(sentence) - sequence_length):
    # 10 characters = 1 training sequence
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    # convert x, y str to index(int)
    x_idx = [char_dic[c] for c in x_str] 
    y_idx = [char_dic[c] for c in y_str] 
    # append to dataset list
    dataX.append(x_idx)
    dataY.append(y_idx)

    # monitoring
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x_idx, y_idx))

[  0/360] [if you wan]=>[f you want]
          [14, 20, 12, 23, 19, 7, 12, 1, 9, 22]=>[20, 12, 23, 19, 7, 12, 1, 9, 22, 15]
[  1/360] [f you want]=>[ you want ]
          [20, 12, 23, 19, 7, 12, 1, 9, 22, 15]=>[12, 23, 19, 7, 12, 1, 9, 22, 15, 12]
[  2/360] [ you want ]=>[you want t]
          [12, 23, 19, 7, 12, 1, 9, 22, 15, 12]=>[23, 19, 7, 12, 1, 9, 22, 15, 12, 15]
[  3/360] [you want t]=>[ou want to]
          [23, 19, 7, 12, 1, 9, 22, 15, 12, 15]=>[19, 7, 12, 1, 9, 22, 15, 12, 15, 19]
[  4/360] [ou want to]=>[u want to ]
          [19, 7, 12, 1, 9, 22, 15, 12, 15, 19]=>[7, 12, 1, 9, 22, 15, 12, 15, 19, 12]


In [6]:
# check type and data
print('\n')
print ((type(dataX)))
print (dataX[0])
print (dataX[1])
print (dataX[168])
print (dataX[169])

print('\n')
print ((type(dataY)))
print (dataY[0])
print (dataY[1])
print (dataY[168])
print (dataY[169])



<class 'list'>
[14, 20, 12, 23, 19, 7, 12, 1, 9, 22]
[20, 12, 23, 19, 7, 12, 1, 9, 22, 15]
[12, 19, 20, 12, 15, 6, 0, 12, 13, 0]
[19, 20, 12, 15, 6, 0, 12, 13, 0, 9]


<class 'list'>
[20, 12, 23, 19, 7, 12, 1, 9, 22, 15]
[12, 23, 19, 7, 12, 1, 9, 22, 15, 12]
[19, 20, 12, 15, 6, 0, 12, 13, 0, 9]
[20, 12, 15, 6, 0, 12, 13, 0, 9, 8]


### 입력문자 list를 3-dim로, 타겟문자 list를 2-dim로 변환  

In [7]:
# input tensor 생성 
X = np.array(dataX)                                      # ndarray(170,10)<-[170,10]
# convert list sequence to 3dim array(one-hot coding)
sequences = [tf.keras.utils.to_categorical(
                 x, num_classes = data_dim) for x in X]  # list[170,10,25]
X = np.array(sequences)                                  # ndarray(170,10,25)

# Target tensor 생성
y = np.array(dataY)[:,-1]                                # ndarray(170,)<-[170,10]
# convert the vector to 2-dim 
#y = np.array(dataY) 
y = tf.keras.utils.to_categorical(
                  y, num_classes = data_dim)             # (170,25)

print (y.shape) # (170, 25)
print (y[0])    # t -> one-hot ,  (if you wan't')

(350, 25)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


## **Model**

### Define

In [10]:
# Sequenctial model define: LSTM + Dense
hidden_dim = 75
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=hidden_dim,
                               input_shape=(sequence_length,num_classes) ,return_sequences=True))
model.add(tf.keras.layers.LSTM(units=hidden_dim))
model.add(tf.keras.layers.Dense(data_dim, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 75)            30300     
_________________________________________________________________
lstm_3 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense (Dense)                (None, 25)                1900      
Total params: 77,500
Trainable params: 77,500
Non-trainable params: 0
_________________________________________________________________


### Compile

In [0]:
model.compile(loss='categorical_crossentropy', # one-hot coding
              optimizer='adam', 
              metrics=['accuracy'])

### Fit

Epoch 1000/1000   
170/170 [==============================] - 0s 235us/sample - loss: 0.0012 - accuracy: 1.0000    
CPU times: user 48.3 s, sys: 6.54 s, total: 54.8 s    
Wall time: 47.8 s (@Notebook Setting/GPU)

In [12]:
%%time
model.fit(X,            # X.shape : (170, 10, 25)
          y,            # y.shape : (170, 25)
          epochs=200)

Train on 350 samples
Epoch 1/200
350/350 [==============================] - 4s 11ms/sample - loss: 3.1959 - accuracy: 0.1200
Epoch 2/200
350/350 [==============================] - 0s 167us/sample - loss: 3.0338 - accuracy: 0.1886
Epoch 3/200
350/350 [==============================] - 0s 168us/sample - loss: 2.9075 - accuracy: 0.1886
Epoch 4/200
350/350 [==============================] - 0s 168us/sample - loss: 2.8609 - accuracy: 0.1886
Epoch 5/200
350/350 [==============================] - 0s 181us/sample - loss: 2.8392 - accuracy: 0.1886
Epoch 6/200
350/350 [==============================] - 0s 212us/sample - loss: 2.8387 - accuracy: 0.1886
Epoch 7/200
350/350 [==============================] - 0s 189us/sample - loss: 2.8186 - accuracy: 0.1886
Epoch 8/200
350/350 [==============================] - 0s 174us/sample - loss: 2.7816 - accuracy: 0.1886
Epoch 9/200
350/350 [==============================] - 0s 177us/sample - loss: 2.7390 - accuracy: 0.1886
Epoch 10/200
350/350 [=============

## **Generate Text:** Analysis

### Define generate_seq function

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, char_dic, seq_length, seed_text, n_chars):
	in_text = seed_text									# seed text + generated text
	# generate a fixed number of characters
	for i in range(n_chars):
		# encode the characters as integers
		encoded = [char_dic[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre') #  원하는 사이즈만큼 가져와서 그것만 남겨둔다. 
		encoded = tf.keras.utils.to_categorical(encoded, num_classes=len(char_dic))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		for char, index in char_dic.items():
			if index == yhat:
				break
		# append to input
		in_text += char
	return in_text

### Generate a sequence of characters

학습문장: if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [17]:
# test start of rhyme
print('Trigger characters: "want to bu "\nResult: \n"{}"'.format( 
      generate_seq(model, char_dic, 10, 'want to bu', 200)))
# test mid-line
print('\nTrigger characters: "collect wo"\nResult: \n"{}"'.format(
      generate_seq(model, char_dic, 10, 'collect wo', 100)))
# test not in original
print('\nTrigger characters: "rather tea"\nResult: "{}"'.format(
      generate_seq(model, char_dic, 10, 'rather than', 200)))

Trigger characters: "want to bu "
Result: 
"want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.if you want to build a ship, don't dr"

Trigger characters: "collect wo"
Result: 
"collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of "

Trigger characters: "rather tea"
Result: "rather thanh lee    i tsiu    t aepp d   eoeodut  tynnubl  r  bbl eeeeusssst   mentnaa   iibh oo     ndddadpuussst,ooopetttttn hs     k 'nddpduuum tttobllll    r rrppdeeaaaccruntmooohtpl lggnlleehhhhhisoy gggyyt"


## **실습과제**

### 과제 1. 더 긴문장을 생성하도록 해보자
####-- 학습문장이 " ...  immensity of the sea."로 끝났다. 첫번째 생성문장이 뒷부분이 이상하다. 180자로 학습한 한계 때문일까? 
#### -- 학습문장 전체를 한번더 반복하여 학습문장을 360자로 확장하고 결과를 비교해 보자



#### -- 결과를 분석하자: 






1번의 경우 180자 이후부터 다시 처음부터 문장이 시작되는 것을 확인할 수 있다.

결과적으로 기존의 가장 끝 단어가 나타난 이후 모델이 학습한 것은 다시 처음부터 이어지는 문장이다. 그렇기 때문에 내부에서 LSTM 을 통해서 학습하는 과정에서 . 이후의 다음 문장은 복사 붙여넣기 한 첫 if가 나오는 것이라고 학습하고, 이를 학습한대로 뱉어낸 것이라 생각된다.

### 과제 2. 세번째 생성문장은 여전히 문제가 많다. 모델을 개선해 보자 
####-- LSTM layer를 추가해 보자 
####-- 더 무엇을 해볼까? 생각해 보자
####-- 결과를 분석하자:

LSTM layer를 추가한다고 좋은 결과가 나오는 것은 아니라는 사실을 위에 새롭게 layer를 더해서 수정한 코드를 보면 알 수 있다.

이유는 지금의 코드는 앞전의 글자들이 나올 때 마지막 글자가 어떤 글자가 나오는 것이 좋은지 그 확률을 계산하고 있다.

즉 전체 시퀀스, 각 글자간 어떻게 연결되고 있는지를 확인하고 있는 것이 아니기 때문에 문장이 구성되는 것을 신경쓰지 않고 아무 글자나 해당 위치에 나오기 가장 적합한 글자를 뱉어내고 있는 것이다.

이를 해결하기 위해서는 시퀀스의 셀들이 모두 결과를 꺼내도록 해야한다. 즉, return_sequences를 True로 바꿔야 하는 것이다.